In [2]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sheth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sheth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv('processed_data.csv')
df['clean_text'] = df['clean_text'].fillna('')
df

C:\Users\sheth\AppData\Local\Temp\ipykernel_7800\3815043307.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('processed_data.csv')


,title,text,score,upvote_ratio,upvotes,Unnamed: 5,Unnamed: 6,text_length_words,clean_text
0,Looking for married Muslim men who have hijabi...,Don’t message me if you can’t live verify. Too...,1,1,1,NaN,NaN,17,dont message cant live verify many scammers tr...
1,Share your istikhara success stories. I need s...,Salaam everyone. I’m a F currently going throu...,1,1,1,NaN,NaN,144,salaam everyone im f currently going divorce i...
2,Fate?,"In the Qur'an, I saw verses in these cases tha...",1,1,1,NaN,NaN,73,quran saw verses cases say dont happy come don...
3,Good Thrift shop find. Highly reccomend,Holocaust book about family of Jewish Hungaria...,1,1,1,NaN,NaN,16,holocaust book family jewish hungarian dwarfs ...
4,Wearing my kippah with tattoos,Shalom friends!\n\nI’m a Baal teshuva with man...,1,1,1,NaN,NaN,68,shalom friends im baal teshuva many tattoos ar...
...,...,...,...,...,...,...,...,...,...
92810,Kosher symbols in the UK,This may be a silly question but I live in Can...,5,0.78,5,NaN,NaN,115,may silly question live canada friend sent bun...
92811,How am I supposed to get to know her?,\nSalam. I am currently speaking to this girl ...,2,1,2,NaN,NaN,187,salam currently speaking girl purpose marriage...
92812,Why is calling for BDS on Israel racist but no...,"Every few months, you have politicians in the ...",11,0.6,11,NaN,NaN,103,every months politicians west across almost pa...
92813,r/LateStageCapitalism,"Comments on this post are wild, so many people...",119,0.98,119,NaN,NaN,23,comments post wild many people subreddit blata...


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:
from simpletransformers.classification import ClassificationModel
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [6]:
# model = ClassificationModel(
#     model_type="bert",
#     model_name="bert-base-uncased",
#     use_cuda=False,  # or False if not using GPU
#     num_labels=1,  # For regression, this should be 1
#     args={
#         "regression": True,  # Set explicitly for regression
#         "num_train_epochs": 5,
#         "learning_rate": 2e-5,
#         "overwrite_output_dir": True,
#         "reprocess_input_data": True,
#         "train_batch_size": 16,
#         "eval_batch_size": 16,
#         "max_seq_length": 256
#     }
# )

In [7]:
df['score'] = pd.to_numeric(df['score'], errors='coerce')
df = df.dropna(subset=['score', 'text']).reset_index(drop=True)

In [8]:
df = df[['clean_text', 'score']]

In [ ]:
df = df[df['clean_text'].str.strip() != '']
print(df['clean_text'].str.strip().eq('').sum())

0


In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
mse_scores = []

In [ ]:

sample_df = df.sample(n=1000, random_state=5826).reset_index(drop=True)
for fold, (train_idx, val_idx) in enumerate(kf.split(sample_df)):
    print(f"\nFold {fold + 1}:")

    train_df = sample_df.iloc[train_idx]
    val_df = sample_df.iloc[val_idx]

    model = ClassificationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=False, 
        num_labels=1, 
        args={
            "regression": True,
            "num_train_epochs": 2,
            "learning_rate": 2e-5,
            "overwrite_output_dir": True,
            "reprocess_input_data": True,
            "train_batch_size": 16,
            "eval_batch_size": 16,
            "max_seq_length": 256,
            "save_model_every_epoch": False,
            "save_eval_checkpoints": False,
        },
    )

    model.train_model(train_df, args={"text": "clean_text", "labels": "score"})

    predictions, raw_outputs = model.predict(val_df['clean_text'].tolist())

    mse = mean_squared_error(val_df['score'], predictions)
    mse_scores.append(mse)

    print(f"Fold {fold + 1} MSE: {mse}")

avg_mse = sum(mse_scores) / len(mse_scores)
print(f"\nAverage MSE across 10 folds: {avg_mse}")



Fold 1:


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\sheth\anaconda3\Lib\site-packages\simpletransformers\classification\classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_256_1_2


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

...

Running Epoch 2 of 2:   0%|          | 0/57 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


0it [00:00, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Fold 10 MSE: 1094.1464842771982

Average MSE across 10 folds: 929.0571088797655


In [12]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state= 5826)

In [ ]:
final_model = ClassificationModel(
    model_type="bert",
    model_name="bert-base-uncased",
    use_cuda=False,
    num_labels=1,
    args={
        "regression": True,
        "num_train_epochs": 3,
        "learning_rate": 2e-5,
        "overwrite_output_dir": True,
        "reprocess_input_data": True,
        "train_batch_size": 32,
        "eval_batch_size": 32,
        "max_seq_length": 256,
        "save_model_every_epoch": True,
        "save_eval_checkpoints": False,
    },
)

final_model.train_model(train_df, args={"text": "clean_text", "labels": "score"})

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\sheth\anaconda3\Lib\site-packages\simpletransformers\classification\classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/147 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_256_1_2


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/2308 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/2308 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/2308 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


(6924, 14876.999921942279)

In [ ]:
from sklearn.metrics import mean_squared_error
import os

epoch_2_model_path = r"C:\Users\sheth\OneDrive\Desktop\Sem 3\NLP\Assignment 2\outputs\checkpoint-4616-epoch-2"

epoch_3_model_path = r'C:\Users\sheth\OneDrive\Desktop\Sem 3\NLP\Assignment 2\outputs\checkpoint-6924-epoch-3'

test_data = test_df[["clean_text", "score"]]

def evaluate_model(model_path, test_data):
    model = ClassificationModel(
        model_type="bert",
        model_name=model_path,
        use_cuda=False,
        num_labels=1,
        args={
            "regression": True,
            "eval_batch_size": 32,
            "max_seq_length": 256,
        },
    )

    predictions, raw_outputs = model.predict(test_data["clean_text"].tolist())

    mse = mean_squared_error(test_data["score"], predictions)
    print(f"Model at {model_path} - MSE: {mse}")
    return mse

print("Evaluating model saved after epoch 2:")
mse_epoch_2 = evaluate_model(epoch_2_model_path, test_data)

print("Evaluating model saved after epoch 3:")
mse_epoch_3 = evaluate_model(epoch_3_model_path, test_data)


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


Evaluating model saved after epoch 2:


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/577 [00:00<?, ?it/s]

Model at C:\Users\sheth\OneDrive\Desktop\Sem 3\NLP\Assignment 2\outputs\checkpoint-4616-epoch-2 - MSE: 8123.19278911996
Evaluating model saved after epoch 3:


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/577 [00:00<?, ?it/s]

Model at C:\Users\sheth\OneDrive\Desktop\Sem 3\NLP\Assignment 2\outputs\checkpoint-6924-epoch-3 - MSE: 8133.221771532237
